Cryptography surrounds us. Every time we access an “https” website, we are using incredibly powerful cryptographic tools. This class will take a high-level look at what tools we have at our disposal to keep our communications secure. We will also take a deeper look at asymmetric key cryptography.

https://github.com/LogstonEducation/PDL-Crypto

## What is Cryptography?

In plain english:

The act of taking some information and obscuring it in someway such that we can retreive the original message.

In arrows:

Plain text -> Cipher text -> Plain text

In math:

- Fe(PT) = CT   (encryption function)
- Fd(CT) = PT   (decryption function)

Fe and Fd are ciphers. 

#### Ceaser Cipher

https://en.wikipedia.org/wiki/Caesar_cipher (~75 BCE)

In [2]:
import string

alphabet = string.ascii_lowercase

In [3]:
alphabet

'abcdefghijklmnopqrstuvwxyz'

In [4]:
alphabet.index('p')

15

In [6]:
alphabet[15 + 3]

's'

In [ ]:
# 'hi' -> 8, 9

# + 13 

# 21, 22
# 'uv'

In [12]:
# Complete in class...
import string

alphabet = string.ascii_lowercase

def cipher(plaintext: str) -> str: # rot-13
    ciphertext = []
    
    for char in plaintext:
        if char == ' ':
            ciphertext.append(char)
            continue
        
        # char = h -> 8
        # 8 -> 8 + 13 = 21
        # alphabet[21] cipher char
        cipher_char = alphabet[(alphabet.index(char) + 13) % len(alphabet)]

        ciphertext.append(cipher_char)
    
    return ''.join(ciphertext)

In [13]:
cipher('hi which way did they go')

'uv juvpu jnl qvq gurl tb'

In [14]:
cipher('uv juvpu jnl qvq gurl tb')

'hi which way did they go'

In [11]:
import string

alphabet = string.ascii_lowercase

def cipher(plaintext: str, rot: int=13):
    # clean plain text
    plaintext = plaintext.lower()

    new_string = []
    for char in plaintext:
        if char in alphabet:
            offset = alphabet.index(char)
            new_offset = (offset + rot) % len(alphabet)
            char = alphabet[new_offset]
            
        new_string.append(char)

    return ''.join(new_string)

In [12]:
ciphertext = cipher('everything should be made as simple as possible. but not simpler.')
ciphertext

'rirelguvat fubhyq or znqr nf fvzcyr nf cbffvoyr. ohg abg fvzcyre.'

In [13]:
cipher(ciphertext)

'everything should be made as simple as possible. but not simpler.'

In [14]:
ciphertext = cipher('everything should be made as simple as possible. but not simpler.', 6)
ciphertext

'kbkxeznotm ynuarj hk sgjk gy yosvrk gy vuyyohrk. haz tuz yosvrkx.'

In [15]:
cipher(ciphertext, -6)

'everything should be made as simple as possible. but not simpler.'

## Binary Representation and XOR

Basic Binary Arithmetic 
```
- 1 AND 1 => 1
- 1 AND 0 => 0
- 0 AND 0 => 0
---
- 1 OR 1 => 1
- 1 OR 0 => 1
- 0 OR 0 => 0
---
- 1 XOR 1 => 0
- 1 XOR 0 => 1
- 0 XOR 0 => 0
```

In [15]:
k = 0b10101010
m = 0b00000010

print('k', format(k, '#010b'))
print('m', format(m, '#010b'))

c = k ^ m  # k is encryption key, m could be plain text, c is ciphertext

print('c', format(c, '#010b'))

k 0b10101010
m 0b00000010
c 0b10101000


In [16]:
m2 = k ^ c  # We can get m back by XORing c with k
print(' m', format(m, '#010b'))
print('m2', format(m2, '#010b'))
print(m == m2, '- messages are equal')

 m 0b00000010
m2 0b00000010
True - messages are equal


In [18]:
k2 = c ^ m  # We can get k back by XORing c with m
print(' k', format(k, '#010b'))
print('k2', format(k2, '#010b'))
print(k == k2, '- keys are equal')

 k 0b10101010
k2 0b10101010
True - keys are equal


Let's pretend "k" is our secret series of bits and "m" is our message. It's pretty easy for a hacker to get the message back from this series of bits just by trying all 256 bit options that "k" could take on. 

So how do we keep things secret? 

- We increase the size of the message/key.
- We pass the message through several XORs of bits so that a single XORing would not return anything meaning full.
- We XOR it with a completely random bits. (both of the above depend on this)

At a certain point trying all combinations of bits and XOR becomes computationally infeasible with current computing power. At that point (which is itself very vauge, different for different situations) we say we are satisfied.

So where do we get a bunch of random 1s and 0s? A good psudeo-random number generator (PRNG). 

We build those shooting for these goals (BSI evaluation criteria, [1]):

- Any two sequences should be different from each other.
- Sequences must be indistinguishable from 'true random' numbers according to specified statistical tests.
- Can't reliably guess a bit from any previous or future bits.
- Can't reliably guess a previously or future generatred sequence if you know the internal state of the PRNG for a current sequence.

PRNG List: https://en.wikipedia.org/wiki/List_of_random_number_generators#Pseudorandom_number_generators_(PRNGs)

Mersenne Twister

The Mersenne Twister is probably the most common PRNG in use. Used for Python's `random` library.

https://en.wikipedia.org/wiki/Mersenne_Twister

Refs:

- [1] https://en.wikipedia.org/wiki/Pseudorandom_number_generator#BSI_evaluation_criteria


## Stream Ciphers & Block Ciphers

###### Block Ciphers 

Chop a **known** length message (plaintext) into equal sized parts (commonly 64, 128, 265 bits) and run each part through the cipher. 

- Simple
- Reversing encryption is hard

###### Stream Ciphers 

Take a message of **unknown** length message (plaintext) and chop it up byte by byte as you get the message. Convert one byte at a time to cipher text. Change the 1s and 0s so that its not the same XOR every n bytes.

- Only option for unknown length messages
- Complex
- Easier to crack

A stream of pefectly random bits that's the exact length of the message we want to encrypt would be called a One-Time pad. 

This is great in theory. However, how would we get the secret One-Time pad to our message recipient? If we accomplished that, we have a problem of size. Each message would be at least double the size of its contents; 50% for message, 50% for key.

Stream ciphers usually use short key to initialize themselves and then the stream cipher produces a "random" looking sequence of bits of infanite length.


Common Ciphers [2]:

- Stream Ciphers: Salsa20, ChaCha, ISAAC, HC-128 and RC4.
- Block Ciphers: AES, TwoFish, Serpent and Camellia

Refs:
- [2] https://en.wikipedia.org/wiki/List_of_random_number_generators

## Public Key Cryptography

So far, these encryption tools use the same key to encrypt as they do to decrypt. If we want to send someone an encrypted message, we need them to have the same key we have. How do we get it to them securely? We have a chicken and egg problem? We both need a shared secret key in order to share a secret key.

Enter Asymmetric Key Cryptography.

We break the problem in half and we only consider sending a message securely to one person. 

A wants to send a message to B securely. B can send anything to A and it does not need to be encrypted. 

What if B sent A a key that twisted a message up and now matter how many times you used the key on the message you would only tighten the encryption on the message. Basically, this key only turns one way. It could be shared with everyone and no one would be able to decrypt messages that were encrypted with it.

Now we just need a key that turns in the opposite direction. 

These two opposite turning keys are actually created at the same time and are often based on two large prime numbers. Because they are both required for the full encrypt -> decrypt flow, we call them two halves of the same key. B will send A her public half, A will encrypt the message with that half, and B will decrypt it with her other half. 

Browsers are shipped with the public key's of the major Certificate Authorties (entities that certify other people's public keys) and by virtue of the fact that we can trust these CA's public keys and they say that this is "Amazon's" public key, we can trust we have amazon's public key. Thus we can encrypt messages bound for Amazon.com. This allows us to keep our CC #'s secret on the way to Amazon.com. How does Amazon.com send us information back securely? 

As it turns out, doing the public key crypto stuff is slow. So instead, it is only used at the beginning of accessing a page. From there, both client and server decide on a shared secret one time key. This key is then used with a block or stream cipher to encrypt the rest of the message. 

RSA Paper: https://people.csail.mit.edu/rivest/Rsapaper.pdf

### GPG

###### https://www.gnupg.org/download/index.html#binary

```
$ gpg --gen-key
gpg (GnuPG/MacGPG2) 2.2.17; Copyright (C) 2019 Free Software Foundation, Inc.
This is free software: you are free to change and redistribute it.
There is NO WARRANTY, to the extent permitted by law.

Note: Use "gpg --full-generate-key" for a full featured key generation dialog.

GnuPG needs to construct a user ID to identify your key.

Real name: Paul Logston
Email address: paul.logston@columbia.edu
You selected this USER-ID:
    "Paul Logston <paul.logston@columbia.edu>"

Change (N)ame, (E)mail, or (O)kay/(Q)uit? o
We need to generate a lot of random bytes. It is a good idea to perform
some other action (type on the keyboard, move the mouse, utilize the
disks) during the prime generation; this gives the random number
generator a better chance to gain enough entropy.
We need to generate a lot of random bytes. It is a good idea to perform
some other action (type on the keyboard, move the mouse, utilize the
disks) during the prime generation; this gives the random number
generator a better chance to gain enough entropy.
gpg: key 5FE8521885FD29FF marked as ultimately trusted
gpg: directory '/Users/paul/.gnupg/openpgp-revocs.d' created
gpg: revocation certificate stored as '/Users/paul/.gnupg/openpgp-revocs.d/DFFE9ACF48C871FA60821DD45FE8521885FD29FF.rev'
public and secret key created and signed.

pub   rsa2048 2019-11-04 [SC] [expires: 2021-11-03]
      DFFE9ACF48C871FA60821DD45FE8521885FD29FF
uid                      Paul Logston <paul.logston@columbia.edu>
sub   rsa2048 2019-11-04 [E] [expires: 2021-11-03]
```

---

```
$ gpg --list-keys                                                          
/Users/paul/.gnupg/pubring.kbx
------------------------------
pub   rsa2048 2019-11-04 [SC] [expires: 2021-11-03]
      DFFE9ACF48C871FA60821DD45FE8521885FD29FF
uid           [ultimate] Paul Logston <paul.logston@columbia.edu>
sub   rsa2048 2019-11-04 [E] [expires: 2021-11-03]
```

---

```
$ echo "HI" > secret.txt
$ cat secret.txt 
HI     
```

---

```
$ gpg --encrypt --recipient DFFE9ACF48C871FA60821DD45FE8521885FD29FF secret.txt
```

---

```
$ less secret.txt.gpg
```

---

```
$ gpg --output secret.out.txt --decrypt secret.txt.gpg
```

---

```
$ gpg --delete-secret-key  DFFE9ACF48C871FA60821DD45FE8521885FD29FF 
gpg (GnuPG/MacGPG2) 2.2.17; Copyright (C) 2019 Free Software Foundation, Inc.
This is free software: you are free to change and redistribute it.
There is NO WARRANTY, to the extent permitted by law.

sec  rsa2048/5FE8521885FD29FF 2019-11-04 Paul Logston <paul.logston@columbia.edu>

Delete this key from the keyring? (y/N) y
This is a secret key! - really delete? (y/N) y
```

---

```
$ gpg --delete-key  DFFE9ACF48C871FA60821DD45FE8521885FD29FF        
gpg (GnuPG/MacGPG2) 2.2.17; Copyright (C) 2019 Free Software Foundation, Inc.
This is free software: you are free to change and redistribute it.
There is NO WARRANTY, to the extent permitted by law.

pub  rsa2048/5FE8521885FD29FF 2019-11-04 Paul Logston <paul.logston@columbia.edu>

Delete this key from the keyring? (y/N) y
```

### Keybase.io

https://keybase.io/logston

Uses https://saltpack.org/ rather than PGP (which is used by gpg).


### Signal

https://signal.org/

Open source. No one can add logic that the public can not see. 

### What's App

Not open source. Owned by Meta (aka. Facebook).

### Tor

https://www.torproject.org/download/

### VPNs and WireGuard
https://www.wireguard.com/

### Fun Coding Challenges
https://cryptopals.com/